In [11]:
import easyocr
import requests
import cv2
import numpy as np
import matplotlib.pyplot as plt
import ssl
from io import BytesIO
from PIL import Image
import re

In [12]:
ssl._create_default_https_context = ssl._create_unverified_context

In [13]:
reader = easyocr.Reader(['en'])

/Users/pranayobla/Desktop/Projects/amazon-hackathon/venv/lib/python3.11/site-packages/easyocr/detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_d

In [14]:
def download_image(url):
    response = requests.get(url)
    if response.status_code == 200:
        img_data = response.content
        img = Image.open(BytesIO(img_data))
        return img
    else:
        raise Exception(f"Failed to download image. Status code: {response.status_code}")

In [15]:
def extract_text_from_url(image_url):
    img = download_image(image_url)
    img_cv = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    img_path = 'temp_image.jpg'
    img.save(img_path)
    result = reader.readtext(img_path)
    text_array = [text for (_, text, confidence) in result if confidence >= 0.5]
    lowercase_text_array = [text.lower() for text in text_array]
    return lowercase_text_array

In [16]:
image_url = 'https://m.media-amazon.com/images/I/81xsq6vf2qL.jpg'
text = extract_text_from_url(image_url)
text

['horbaach',
 'directions:  for adults, take two (2) vegan capsules',
 'daily, preferably with a meal. do not exceed stated dose',
 'nutrition information',
 'high',
 'typically per daily dose',
 'psyllium husk powder',
 '1400mg',
 'psyllium ',
 'ingredients: psyllium husk powder , capsule shell',
 'droxypropylmethylcellulose) , anti-caking agents (mag-',
 'nesium salts of fatty acids, silicon dioxide):',
 'may contain sesame seeds & mustard: for allergens,',
 'husk',
 'see the ingredients in bold:',
 'notice: take this product with 22oml of fluids taking',
 'this product without adequate fluid may cause the pos -',
 'plantago ovata',
 'sibility of choking: do not use this product if you have',
 '140omg',
 'difficulty swallowing: if you experience chest pain, vom-',
 'plant seeds',
 'iting or difficulty in swallowing or breathing after taking',
 'this product; seek immediate medical attention. do not',
 'take before sleeping: fibre products can affect the ab-',
 'sorption of many medic

In [17]:
def contains_number(s):
    return bool(re.search(r'\d', s))

In [18]:
def remove_special_characters(s):
    return re.sub(r'[^a-zA-Z0-9\s]', '', s)

In [19]:
processed_text_array = [remove_special_characters(t) for t in text if contains_number(t)]
processed_text_array

['directions  for adults take two 2 vegan capsules',
 '1400mg',
 'notice take this product with 22oml of fluids taking',
 '140omg',
 'within 15 hours of taking medication',
 '365',
 'hu16790 ']